**Class 2: Model-based policy optimization**

1. [Everything you need to know](#everything)
2. [Reminder](#reminder)
3. [Value Iteration](#vi)
4. [Policy Iteration](#pi)
5. [Policy optimization with linear programming](#lp)
6. [Asynchronous Dynamic Programming](#adp)
    1. [Asynchronous Value Iteration](#avi)
    2. [Asynchronous Policy Iteration](#api)

The previous class introduced the MDP model which is the foundation upon which RL theory is built, and characterized optimal control policies. In this class, we investigate how one can use the model algorithmically to find optimal policies.

# <a id="everything"></a>Everything you need to know


Everything you should remember after this session.
<div class="alert alert-success">
<ul>
<li> Value Iteration: finding $V^*$ or $Q^*$ by repeatedly applying $T^*$ to any initial function.
<li> Policy Iteration: finding $V^*$ or $Q^*$ and $\pi^*$ by building the sequence $\pi_{n+1}(s) = \arg\max_{a\in A} Q^{\pi_n}(s,a)$ that converges to $\pi^*$. Repeatedly alternates an evaluation and an improvement phase.
</ul>
</div>


# <a id="reminder"></a>Reminder

Recall the main results from the previous class.

We have introduced the general **discrete-time stochastic optimal control problem**:
- Environment (discrete time, non-deterministic, non-linear, Markov) $\leftrightarrow$ MDP.
- Behaviour $\leftrightarrow$ control policy $\pi : s\mapsto a$.
- Policy evaluation criterion $\leftrightarrow$ $\gamma$-discounted criterion.
- Goal $\leftrightarrow$ Maximize value function $V^\pi(s)$, $Q^\pi(s,a)$.
- Evaluation eq. $\leftrightarrow$ $V^\pi = T^\pi V^\pi$, $Q^\pi = T^\pi Q^\pi$, linear equation, contraction mapping.
- Bellman optimality eq. $\leftrightarrow$ $V^* = T^* V^*$, $Q^* = T^* Q^*$, non-linear equation, contraction mapping.

Based on the analysis of the previous section, we introduce several algorithms that will lay a solid ground for model-free RL algorithms in the next class.

# <a id="vi"></a>Value Iteration

Value iteration is actually the algorithm we defined in the `vf_optim` function in the correction of the previous class' last exercice. It directly exploits the contraction mapping property of $T^*$ and iterates over value functions in order to converge to $V^*$. Once, $V^*$ is found, finding the optimal policy is a matter of writing $Q^*$ and defining a policy that is $Q^*$-greedy.<br>

<div class="alert alert-warning"><b>Exercice:</b><br>
Write the pseudo-code of Value Iteration.
</div>

<div class="alert alert-danger"><a href="#answersVI" data-toggle="collapse"><b>Answer:</b></a><br>
<div id="answersVI" class="collapse">
    Input data: $V$, $\epsilon$<br>
    Init: $\Delta = \epsilon+1$<br>
    While $\Delta \geq \epsilon$:<br>
    &nbsp;&nbsp;&nbsp; For $s \in S$:  <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; For $a\in A$:  <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $Q(s,a) = r(s,a) + \gamma \sum_{s'} p(s'|s,a) V(s')$ <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $V_{new}(s) = \max_a Q(s,a)$  <br>
    &nbsp;&nbsp;&nbsp; $\Delta = \| V_{new}-V \|_\infty$  <br>
    &nbsp;&nbsp;&nbsp; $V = V_{new}$  <br>
    Return $V$  <br>
</div>
</div>

<div class="alert alert-warning"><b>Exercice:</b><br>
Compute and display an optimal policy for the FrozenLake game, using Value Iteration. If you copy-paste the results of exercices 2, 3 and 8 of the previous class, you almost don't need to write any new code.
</div>

In [ ]:
# %load solutions/RL2_exercice1.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).


Those familiar with the principles of Dynamic Programming will note that Value Iteration is a Dynamic Programming algorithm that operates in value function space, monotonically hopping from value function to value function.

<div class="alert alert-warning"><b>Exercice:</b><br>
What is the time complexity of Value Iteration in terms of $|S|$ and $|A|$?
</div>

<div class="alert alert-danger"><a href="#answersComplexVI" data-toggle="collapse"><b>Answer:</b></a><br>
<div id="answersComplexVI" class="collapse">
$O(S^2 A)$
</div>
</div>

# <a id="pi"></a>Policy Iteration

The Policy Iteration algorithm stems from the following remark. Suppose we have a policy $\pi$ and know its value function $V^\pi$ and state-action value function $Q^\pi$. Then, the non-stationary policy $\pi'$ that acts greedily with respect to $Q^\pi$ for the first time step and then follows $\pi$ has a value function $V^{\pi'}$ that is greater or equal to $V^\pi$ (equal if $\pi$ is optimal, strictly greater otherwise). Actually, the contraction property of $T^*$ insures that the stationary policy $\pi'$ that is greedy with respect to $Q^\pi$ is at least as good as $\pi$, that is $V^{\pi'}\geq V^\pi$. Consequently, the sequence of policies defined by $\pi_{n+1}(s) = \arg\max_{a\in A} Q^{\pi_n}(s,a)$ has a monotonically improving corresponding sequence of value functions $V^{\pi_n}$ and converges to $\pi^*$.<br>
<br>
Let's make this more simple with a drawing. Policy iteration alternates two phases:
1. Evaluate $\pi_n$ $\rightarrow Q^{\pi_n}$
2. Compute $\pi_{n+1}$ as the $Q^{\pi_n}$-greedy policy

<img src="img/policyiteration.png">

The process above defines a sequence of policies **and** value functions. Since, for finite state and action spaces, the number of policies is finite, Policy Iteration is guaranteed to converge in a finite number of iterations.

However, $\pi_{n+1} = \pi_n$ is not a valid termination criterion for Policy Iteration!

<div class="alert alert-warning"><b>Exercice:</b><br>
Can you explain why? What would be a sound termination criterion?
</div>

<div class="alert alert-danger"><a href="#answersPIstop" data-toggle="collapse"><b>Answer:</b></a><br>
<div id="answersPIstop" class="collapse">
    
The improvement step of policy iteration consists in computing $\pi_{n+1}(s) = \arg\max_a Q_n(s,a)$.  
But there is no reason for the maximizing action to be unique.  
In the case of multiple maximizing actions, the sequence of policies $\pi_n$ could very well oscillate between several equivalent optimal policies.  
This important lesson to keep from this is that the Bellman equation guarantees that $V^*$ is unique, but also that several policies might be optimal (and have value $V^*$).  
    
Also, oscillation between two quasi-equivalent policies might occur when the computation of $Q^{\pi_n}$ carries small errors.  
    

One solution is to define a tie-breaker between equivalent actions. However, even if its a bit more costly computationally, it might be safer to compare $V^{\pi_{n+1}}$ and $V^{\pi_n}$ and allow a precision error of $\epsilon$. To observe this phenomenon, one could compare two versions of Policy Iteration: one that uses the `policy_eval_iter_mat` and one that uses the `policy_eval_lin` functions from exercice 6 of the previous class. The latter carries the errors of matrix inversion which causes the algorithm to never terminate and alternate between equivalent optimal policies.
</div>
</div>

<div class="alert alert-warning"><b>Exercice:</b><br>
Write the pseudo-code of Policy Iteration using the contraction property of $T^\pi$.
</div>

<div class="alert alert-danger"><a href="#answersPI" data-toggle="collapse"><b>Answer:</b></a><br>
<div id="answersPI" class="collapse">
    Input data: $\pi$, $\epsilon$<br>
    Init: $\Delta = \Delta_\pi = \epsilon+1$<br>
    While $\Delta_\pi \geq \epsilon$:<br>
&nbsp;&nbsp;&nbsp; # Policy evaluation, solve $V=T^\pi V$<br>
&nbsp;&nbsp;&nbsp; $V_{old} = V$<br>
&nbsp;&nbsp;&nbsp; While $\Delta \geq \epsilon$: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; For $s \in S$:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $V_{new}(s) = r(s,\pi(s)) + \gamma \sum_{s'} p(s'|s,\pi(s)) V(s')$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\Delta = \| V_{new}-V \|_\infty$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $V = V_{new}$<br>
&nbsp;&nbsp;&nbsp; $\Delta_\pi = \|V - V_{old} \|_\infty$<br>
&nbsp;&nbsp;&nbsp; # Policy improvement<br>
&nbsp;&nbsp;&nbsp; For $s \in S$: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; For $a \in A$: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $Q(s,a) = r(s,a) + \gamma \sum_{s'} p(s'|s,a) V(s')$ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\pi(s) = \max_a Q(s,a)$<br>
    Return $\pi$  <br>
</div>
</div>

<div class="alert alert-warning"><b>Exercice:</b><br>
Compute and display an optimal policy for the FrozenLake game, using Policy Iteration. For the sake of simplicity, use a fixed number of iterations for the resolution of $V=T^\pi V$. Recall that exercice 6 of the previous class provided you with a function that computes the value $V^\pi$ of any policy, that exercice 2 transformed a $V$ function into a $Q$ function, and that exercice 3 picked the greedy policy from a $Q$ function.
</div>

In [ ]:
# %load solutions/RL2_exercice2.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).


<div class="alert alert-warning"><b>Exercice:</b><br>
What is the time complexity of Policy Iteration in terms of $|S|$ and $|A|$?
</div>

<div class="alert alert-danger"><a href="#answersComplexPI" data-toggle="collapse"><b>Answer:</b></a><br>
<div id="answersComplexPI" class="collapse">
$O(|S|^2 |A|)$
    
Policy Iteration has the same time complexity as Value Iteration. In practice, for finite state and action spaces, Policy Iteration converges in a finite number of steps (contrarily to Value Iteration). But each of these steps requires the resolution of $V = T^\pi V$ which is where the real computational cost is.
</div>
</div>

As for Value Iteration, those familiar with Dynamic Programming will remark that Policy Iteration is a Dynamic Programming algorithm in policy space, monotonically hopping from policy to policy.

# <a id="lp"></a>Linear Programming

If this is the first time you read this notebook, this part can be skipped. [Link to next part](#adp)

An alternative way of finding $V^*$ is by casting the optimality equation as a linear optimization problem. This formulation is mainly given for your curiosity and we will not study it any further.<br>
<br>
Recall the optimality equation:
$$\forall s\in S, V(s)=\max\limits_{a\in A} \left\{r(s,a) + \gamma \sum\limits_{s'\in S} p(s'|s,a) V(s')\right\}$$
The key remark to transform this into a linear program is to rephrase it as "$V^*$ is the smallest value that dominates over all policy values". This can be written as:
$$\left\{ \begin{array}{c}
\min \sum\limits_{s\in S} V(s)\\
s.t. \ \forall \pi, \ V \geq T^\pi V
\end{array} \right.$$
"For all $\pi$" means for all possible association $s\leftrightarrow a$, so this can be expanded as:
$$\left\{ \begin{array}{c}
\min \sum\limits_{s\in S} V(s)\\
s.t. \ \forall (s,a)\in S\times A, \quad V(s) - \gamma \sum\limits_{s'\in S} p(s'|s,a)V(s') \geq r(s,a)
\end{array}\right.$$
Which, finally, is a linear program with $|S|$ variables and $|S||A|$ constraints.

# <a id="adp"></a>Asynchronous Dynamic Programming

If this is the first time you read this notebook, this part can be skipped. [Link to next part](#rl)

We have seen that Value Iteration and Policy Iteration are Dynamic Programming algorithms. They follow a path, respectively in value function and in policy space that leads to $V^*$ and $\pi^*$. But we can remark that they both perform *state-wise* operations such as:

- $V(s) \leftarrow \max_{a} r(s,a) + \gamma \sum_{s'} p(s'|s,a) V(s')$ for value iteration
- $\pi(s) \leftarrow \arg\max_a Q^{\pi}(s,a)$ for policy iteration
- $V(s) \leftarrow r(s,\pi(s)) + \gamma \sum_{s'} p(s'|s,\pi(s)) V(s')$ for policy evaluation

These state-wise operations are called Bellman backups. Let's rename them, respectively:
- `BBVopt(V,s): return` $\max_{a} r(s,a) + \gamma \sum_{s'} p(s'|s,a) V(s')$
- `BBpiopt(V,s): return` $\arg\max_a r(s,a) + \gamma \sum_{s'} p(s'|s,a) V(s)$
- `BBVval(V,s): return` $r(s,\pi(s)) + \gamma \sum_{s'} p(s'|s,\pi(s)) V(s')$

Then we can rewrite value iteration as:<br>
`
V(s) = Vinit(s) for all s
while error>epsilon
  for s in S
    W(s) = BBVopt(V,s)
  error = norm(W-V)
  V = W
`

And policy iteration as:<br>
`
while(true)
  V(s) = 0 for all s
  for k=1 to K
    for s in S
      V(s) = BBVval(V,s)
  for s in S
    pi2 BBpiopt(V,s)
  if(pi == pi2)
    stop
  else
    pi = pi2
`

## <a id="avi"></a>Asynchronous Value Iteration

Let's take the pseudo-code of value iteration. Why don't we perform directly `V(s) = BBVopt(V,s)`, instead of relying on the intermediate `W` function? Doing so is actually called **Gauss-Seidl Value Iteration** and it opens the door to a much wider class of algorithms called Asynchronous Value Iteration.<br>
<br>
It is crucial to note that in Gauss-Seidl Value Iteration, the order in which the states are considered for backups greatly affects of rewards are propagated through the state space and how the sequence of value functions converges to $V^*$. But still, in Gauss-Seidl Value Iteration, states are updated once per sweep over the state space.

Why wouldn't we update the value of some states more often than others? Would the overall value function still converge to $V^*$? A very powerful theorem actually states that:
<div class="alert alert-success">As long as every state is visited infinitely often by the `V(s)` $\leftarrow$ `BBVopt(V,s)` operation as time tends to $+\infty$, the value function $V$ converges to $V^*$
</div>

Consequently, we could pick states totally randomly in order to perform Bellman backups on $V$ and $V$ would still converge to $V^*$. Although picking states randomly for that purpose seems like a bad idea, identifying a good ordering for the backups can lead to drastic improvements in convergence speed. This is the key idea of **Asynchronous Value Iteration** and has justified (among other things) the popular **Prioritized Sweeping** and **Real-Time Dynamic Programming** algorithms.

## <a id="api"></a>Asynchronous Policy Iteration

Let's now take the pseudo-code of policy iteration. The evaluation step and the improvement step are clearly separated. But one can note that if we require the evaluation step to have infinite precision, `K` needs to tend to $\infty$. So, realistic implementations of Policy Iteration, as the one we crafted earlier, tolerate some error on $V^\pi$. But could we take an arbitrary value for `K`? Would this still converge? The answer is yes and this algorithm is known as **Modified Policy Iteration**.

But then, can we push it a little further and say that `K=1`? Then in this case, Modified Policy Iteration becomes... Value Iteration (check it by yourself)!

As in the value iteration case, can we update the value or policy of a given state in any ordering? The answer is yes again and the theorem states that:
<div class="alert alert-success"> As long as every state is visited infinitely often by the `V(s)` $\leftarrow$ `BBVval(V,s)`  and the $\pi(s) \leftarrow$ `BBpiopt(V,s)` operations as time tends to $+\infty$, the value function $V$ and the policy $\pi$ converge respectively to $V^*$ and $\pi^*$
</div>

That is the most general framework one can give for **Asynchronous Dynamic Programming** in MDP resolution. It is often called **Asynchronous Policy Iteration**.<br>

<div class="alert alert-warning"><b>Exercice:</b><br>
Instead of reasoning on state value functions $V$, we can work directly with state-action value functions $Q$ and policies $\pi$. Suppose we maintain a memory of a function $Q$ and a policy $\pi$, then we can define two Bellman backups:
<ul>
<li> `BBQ(s,a):` $Q(s,a) \leftarrow r(s,a) + \gamma \sum_{s'} p(s'|s,a) Q(s',\pi(s'))$
<li> `BBpi(s):` $\pi(s) \leftarrow \arg\max_a Q(s,a)$
</ul>
We can note the similarity between `BBQ` and the previous `BBVval` and the one between `BBpi` and the previous `BBpiopt`.  The previous `BBVopt` is then the successive application of `BBQ` and `BBpi`.<br>
Rewrite Value Iteration and Policy Iteration with these two elementary operations. Rephrase the key property of Asynchronous Policy Iteration using these operations.
</div>

<div class="alert alert-danger"><a href="#answers3" data-toggle="collapse"><b>Answers:</b></a><br>
<div id="answers3" class="collapse">
Value Iteration:
<code>
Q(s,a) = Qinit(s,a) for all s,a
while error>epsilon
  for s,a in SxA
    BBQ(s,a)
    BBpi(s)</code>
<br><br>
Policy Iteration:
<code>
while(pi not constant)
  Q(s,a) = 0 for all s,a
  while error>epsilon
    for s,a in SxA
      BBQ(s,a)
  for s in S
    BBpi(s)</code>
<br><br>
Asynchronous Policy Iteration: as long as all $s$ and all $a$ are visited infinitely often for Bellman backups `BBQ` and `BBpi` as time tends to $+\infty$, $Q$ and $\pi$ will tend respectively to $Q^*$ and $\pi^*$ (whatever the ordering of backups).
</div>
</div>